In [1]:
%cd /content/drive/MyDrive/Machine Learning/Final Project

[Errno 2] No such file or directory: '/content/drive/MyDrive/Machine Learning/Final Project'
/content


In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install xlrd

In [5]:
raw_df = pd.read_excel("/content/drive/MyDrive/Machine Learning/Final Project/PS206767-553247439.xlsx")

KeyboardInterrupt: ignored

In [ ]:
raw_df

In [ ]:
raw_df.loc[raw_df["Race (Reported)"] == "other", "Race (Reported)"] = "Other"

In [ ]:
df = raw_df[["Gender", "Race (Reported)", "Age", "Height (cm)", "Weight (kg)", "Diabetes", "Simvastatin (Zocor)", "Amiodarone (Cordarone)","INR on Reported Therapeutic Dose of Warfarin","VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T", "Therapeutic Dose of Warfarin"  ]]

In [ ]:
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


counts = df["Gender"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Gender')
plt.show()

counts = df["Race (Reported)"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Race')
plt.show()

counts = df["Age"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Age')
plt.show()

counts = df["Diabetes"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Diabetes')
plt.show()

counts = df["Simvastatin (Zocor)"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Simvastatin (Zocor)')
plt.show()

counts = df["Amiodarone (Cordarone)"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Amiodarone (Cordarone)')
plt.show()

counts = df["VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T')
plt.show()

#From the bar charts, we can come up with a few conclusions about our data. We have more  males than females. For race, White is predominantly more common. Most of the patients are b/t the ages of 50-80. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.hist(df["Height (cm)"], bins=5)
plt.title('Histogram for Height')
plt.show()

plt.hist(df["Weight (kg)"], bins=5)
plt.title('Histogram for Weight')
plt.show()

plt.hist(df["INR on Reported Therapeutic Dose of Warfarin"], bins=5)
plt.title('Histogram for INR on Reported Therapeutic Dose of Warfarin')
plt.show()



In [ ]:
df.info()

We are going to use regression

#We have to make sure that the categorical values are converted to numerical equivalents. Hence, we will be using LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Race (Reported)'] = le.fit_transform(df['Race (Reported)'])
df['Age'] = le.fit_transform(df['Age'])
df['VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T'] = le.fit_transform(df['VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T'])
df['Diabetes'] = le.fit_transform(df['Diabetes'])
df['Simvastatin (Zocor)'] = le.fit_transform(df['Simvastatin (Zocor)'])
df['Amiodarone (Cordarone)'] = le.fit_transform(df['Amiodarone (Cordarone)'])



df

In [ ]:
df.info()

#Here, I am trying to find the numerical equivalent of the categorical values. 

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# df['Gender'] = le.fit_transform(raw_df['Gender'])

# # Print the mapping of encoded values to original categorical values
# print(dict(zip(le.classes_, range(len(le.classes_)))))


# # To get the original categorical value back for a particular encoded value:
# encoded_value = 1
# original_value = le.inverse_transform([encoded_value])
# print(original_value[0])

#Using simple imputation with mean values to have the same number of entries in each column. This will ensure for a better model.

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
import pandas as pd

# Step 1: Simple Imputation with Mean values
step1_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
step1_impute_mean = step1_imputer.fit_transform(df)
step1_impute_df = pd.DataFrame(step1_impute_mean, columns=df.columns + '_impute')
step1_impute_df = pd.concat([df, step1_impute_df], axis=1)
step1_impute_df

new_df = step1_impute_df.drop(['Gender', 'Race (Reported)', 'Age', 'VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T', 'Height (cm)', 'Weight (kg)', 'Diabetes', 'Amiodarone (Cordarone)', 'INR on Reported Therapeutic Dose of Warfarin', 'Therapeutic Dose of Warfarin', 'Simvastatin (Zocor)'], axis = 1)


In [ ]:
new_df.info()

In [ ]:
new_df

##Removed all of the outliers (except last column)



In [ ]:
import statsmodels.api as sm

outliers = []

    # fit OLS model
    #model = sm.OLS(new_df[col], sm.add_constant(new_df.drop([col, new_df.columns[-1]], axis=1))).fit()
model = sm.OLS(new_df['Therapeutic Dose of Warfarin_impute'], sm.add_constant(new_df.drop(['Therapeutic Dose of Warfarin_impute'], axis=1))).fit()

    # get influence measures
influence = model.get_influence()

    # calculate Cook's distance
(c, p) = influence.cooks_distance

    # identify outliers
col_outliers = np.where(c > 4/len(new_df))[0]
threshold = 4 / len(new_df)


    # store outliers for this column
outliers.append(col_outliers)

# merge outlier indices and remove duplicates
outliers = np.unique(np.concatenate(outliers))

new_df_cook = new_df.drop(outliers[:-1])

# print outliers
print("Outliers:", outliers)

In [ ]:
new_df_cook

#Now, we are going to do visualization on our prepped data!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

counts = new_df["Gender_impute"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Gender')
plt.show()

counts = new_df["Race (Reported)_impute"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Race')
plt.show()

counts = new_df["Age_impute"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Age')
plt.show()

counts = new_df["VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T_impute"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T')
plt.show()

counts = new_df["Diabetes_impute"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Diabetes')
plt.show()

counts = new_df["Simvastatin (Zocor)_impute"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Simvastatin')
plt.show()

counts =new_df["Amiodarone (Cordarone)_impute"].value_counts()
counts.plot(kind='bar')
plt.title('Bar Chart for Amiodarone')
plt.show()





In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.hist(new_df["Height (cm)_impute"], bins=5)
plt.title('Histogram for Height')
plt.show()

plt.hist(new_df["Weight (kg)_impute"], bins=5)
plt.title('Histogram for Weight')
plt.show()

plt.hist(new_df["INR on Reported Therapeutic Dose of Warfarin_impute"], bins=5)
plt.title('Histogram for INR on Reported Therapeutic Dose of Warfarin')
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns

corr_matrix = new_df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
#Do not do feature scaling on outputs(target variable)
X_train, X_test, y_train, y_test = train_test_split(new_df.drop(['Therapeutic Dose of Warfarin_impute'], axis=1), new_df['Therapeutic Dose of Warfarin_impute'], test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)



In [ ]:
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
#Method 1: Linear Regression

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# save the model using joblib
joblib.dump(lr_model, 'linear_regression_model.joblib')

# load the model from file
lr_model_loaded = joblib.load('linear_regression_model.joblib')


In [ ]:
#Cross-Validation Score

from sklearn.model_selection import cross_val_score

# calculate cross-validation score for the linear regression model
lr_cv_scores = cross_val_score(lr_model_loaded, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

print("Cross-validation scores for Linear Regression:", lr_cv_scores)
print('Mean CV score:', lr_cv_scores.mean())

In [ ]:
#Method 2: Lasso 

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# define the parameter grid for Lasso model
lasso_param_grid = {'alpha': [0.01, 0.1, 1, 10]}

# create a Lasso model object
lasso = Lasso()

# create a GridSearchCV object to search over the parameter grid
lasso_grid_search = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, cv=5, scoring='neg_mean_squared_error')

# fit the GridSearchCV object to the training data
lasso_grid_search.fit(X_train, y_train)

# get the best Lasso model and its hyperparameters
best_lasso_model = lasso_grid_search.best_estimator_
best_lasso_model.fit(X_train, y_train)
joblib.dump(best_lasso_model, 'lasso_model.joblib')

filename = 'lasso_model.joblib'
lasso_model = joblib.load(filename)

In [ ]:
from sklearn.model_selection import cross_val_score

# get the cross-validation score for the best Lasso model
lasso_cv_score = cross_val_score(best_lasso_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# print the mean of the cross-validation score
print('Mean CV score:', lasso_cv_score.mean())


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import joblib

# define the parameter grid for the MLPRegressor
mlp_param_grid = {'hidden_layer_sizes': [(50,)],
                  'activation': ['relu'],
                  'alpha': [0.0001],
                  'learning_rate': ['constant']}

# create an MLPRegressor object
mlp = MLPRegressor(max_iter=1000)

# create a GridSearchCV object to search over the parameter grid
mlp_grid_search = GridSearchCV(estimator=mlp, param_grid=mlp_param_grid, cv=5, scoring='neg_mean_squared_error')

# fit the GridSearchCV object to the training data
mlp_grid_search.fit(X_train, y_train)


# get the best MLPRegressor model and its hyperparameters
best_mlp_model = mlp_grid_search.best_estimator_
best_mlp_model.fit(X_train, y_train)
joblib.dump(best_mlp_model, 'mlp_model.joblib')

filename = 'mlp_model.joblib'
mlp_model = joblib.load(filename)


In [ ]:
from sklearn.model_selection import cross_val_score

# get the cross-validation score for the MLPRegressor model
cv_scores = cross_val_score(best_mlp_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# print the mean and standard deviation of the cross-validation scores
print("Cross-Validation Scores:\n", cv_scores)
print("Mean CV Score:", cv_scores.mean())
print("Std CV Score:", cv_scores.std())


In [ ]:
print("Linear Regression accuracy: ", lr_model.score(X_train, y_train))

print("Lasso accuracy: ", best_lasso_model.score(X_train, y_train))

print("MLP train accuracy: ", best_mlp_model.score(X_train, y_train))


In [ ]:
#Metrics for Linear Regression

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# make predictions on the test set
y_pred = lr_model_loaded.predict(X_test)

# calculate the mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# calculate the mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)

# calculate the R-squared (R2) score
r2 = r2_score(y_test, y_pred)

print("MAE for Linear Regression:", mae)
print("MSE for Linear Regression:", mse)
print("R2 score for Linear Regression:", r2)


In [ ]:
#Metrics for Lasso

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# make predictions on the test set
y_pred = lasso_model.predict(X_test)

# calculate the mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# calculate the mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)

# calculate the R-squared (R2) score
r2 = r2_score(y_test, y_pred)

print("MAE for Lasso:", mae)
print("MSE for Lasso:", mse)
print("R2 score for Lasso:", r2)


In [ ]:
#Metrics for MLP

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# make predictions on the test set
y_pred = mlp_model.predict(X_test)

# calculate the mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# calculate the mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)

# calculate the R-squared (R2) score
r2 = r2_score(y_test, y_pred)

print("MAE for MLP:", mae)
print("MSE for MLP:", mse)
print("R2 score for MLP:", r2)

#Based on the tables in the paper, our values are relatively close. Hence, we will move on to the next step.

In [ ]:
#Learning Curve for linear regression

from sklearn.model_selection import learning_curve
import numpy as np
import matplotlib.pyplot as plt

# define the training set sizes to use in the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# calculate the learning curve
train_sizes, train_scores, test_scores = learning_curve(lr_model_loaded, X_train, y_train, train_sizes=train_sizes, cv=5, scoring='neg_mean_squared_error')

# convert the negative MSE scores to positive values
train_scores = -train_scores
test_scores = -test_scores

# calculate the mean and standard deviation of the training and test scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training error')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.plot(train_sizes, test_mean, label='Cross-validation error')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Number of training examples')
plt.ylabel('MSE')
plt.title('Learning curve for Linear Regression')
plt.legend()
plt.show()


In [ ]:
#Learning Curve for Lasso

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(best_lasso_model, X_train, y_train, cv=5)

# Calculate mean and standard deviation for training scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Calculate mean and standard deviation for test scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.plot(train_sizes, train_mean, label='Training score')
plt.plot(train_sizes, test_mean, label='Cross-validation score')

# Draw the bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)

# Add labels and legend
plt.xlabel('Number of training samples')
plt.ylabel('MSE')
plt.legend()

# Show the plot
plt.show()


In [ ]:
#Learning curve for MLP
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(
    mlp_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

train_scores_mean = -np.mean(train_scores, axis=1)
test_scores_mean = -np.mean(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.title('MLPRegressor Learning Curve')
plt.xlabel('Training Examples')
plt.ylabel('Neg MSE')
plt.grid()
plt.plot(train_sizes, train_scores_mean, label='Training error')
plt.plot(train_sizes, test_scores_mean, label='Cross-validation error')
plt.legend()
plt.show()


#Gradio

In [ ]:
# Step 1: install Gradio
!pip install --quiet gradio

In [ ]:
import gradio as gr
import tensorflow as tf
import cv2

input1 = input_module1 = gr.inputs.Textbox(label="Input Features. Please enter them in array form. Please use this template: [Gender, Race, Age, Height (cm), Weight (kg), Diabetes, Simvastatin (Zocor), Amiodarone (Cordarone), INR on Reported Therapeutic Dose of Warfarin, VKORC1 genotype: -1639 G>A (3673); chr16:31015190; rs9923231; C/T]. Please convert your values accordingly: \n {'African-American': 0, 'Asian': 1, 'Black': 2, 'Black African': 3, 'Black Caribbean': 4, 'Black or African American': 5, 'Black other': 6, 'Caucasian': 7, 'Chinese': 8, 'Han Chinese': 9, 'Hispanic': 10, 'Indian': 11, 'Intermediate': 12, 'Japanese': 13, 'Korean': 14, 'Malay': 15, 'Other': 16, 'Other (Black British)': 17, 'Other (Hungarian)': 18, 'Other Mixed Race': 19, 'White': 20, nan: 21} \n {'female': 0, 'male': 1, nan: 2} \n For age: {'10 - 19': 0, '20 - 29': 1, '30 - 39': 2, '40 - 49': 3, '50 - 59': 4, '60 - 69': 5, '70 - 79': 6, '80 - 89': 7, '90+': 8, nan: 9} /n For Diabetes, Simvastatin (Zocor), and Amiodarone (Cordarone): {0.0: 0, 1.0: 1, nan: 2} \n For VKORC1: {'A/A': 0, 'A/G': 1, 'G/G': 2, nan: 3}", lines=10)

input2 = input_module1 = gr.inputs.Dropdown(choices=["Linear Regression", "Lasso", "MLP"], label = "Method")

In [ ]:
import joblib

lr_model_loaded = joblib.load('linear_regression_model.joblib')

filename = 'lasso_model.joblib'
lasso_model = joblib.load(filename)

filename = 'mlp_model.joblib'
mlp_model = joblib.load(filename)

In [ ]:
import numpy as np

def predict(input1, input2):
    # Convert input data to a numpy array
    input_arr = np.array(eval(input1))
    # Reshape input data to match model input dimensions
    input_arr = input_arr.reshape(1, -1)
    
    # Select the appropriate model
    if input2 == "Linear Regression":
        preds = lr_model_loaded.predict(input_arr)
    elif input2 == "Lasso":
        preds = lasso_model.predict(input_arr)
    elif input2 == "MLP":
        preds = mlp_model.predict(input_arr)
    else:
        preds = None

    if preds is not None:
        output_label = preds[0]
    else:
        output_label = "Unknown"
    
    return output_label

label = gr.outputs.Label(label = "Required Amount")


In [ ]:
#[1.0,20.0,5.0,193.04,115.70,2.0,0.0,0.0,2.60,1.0]

In [ ]:
interface = gr.Interface(fn=predict, inputs=[input1, input2], examples=[
        ["[1.0,20.0,5.0,193.04,115.70,2.0,0.0,0.0,2.60,1.0]"]
    ], outputs=label).launch(debug = True, share = True)